# Simulating Many Sources in `castor_etc`

Isaac Cheng - April 2022

This notebook shows how to process many point sources using the `castor_etc` Python
package. Note that we do not yet support generating multiple sources in the same aperture
(but you can always simulate your own and [use your own data](./custom_source.ipynb) with
the ETC!).


In [1]:
import astropy.units as u
import numpy as np
import pandas as pd
from IPython.display import display

from castor_etc.background import Background
from castor_etc.photometry import Photometry
from castor_etc.sources import PointSource
from castor_etc.telescope import Telescope

We will use the same `Telescope` and (sky) `Background` objects for all of our photometry
calculations.


In [2]:
MyTelescope = Telescope()
MyBackground = Background()
MyBackground.add_geocoronal_emission("avg")

Now we will generate many different point sources at different AB magnitudes, then find
the time required (in seconds) to achieve a signal-to-noise ratio of 5 and 10 in each
passband.


In [3]:
TARGET_SNRS = [5, 10]
AB_MAGS = np.arange(22, 29, 0.5)
REDDENING = 0.0

wavelengths = np.arange(90, 1200) * u.nm  # generate the spectrum over these wavelengths

# Make some dictionaries to store the results of our calculations
times_to_one_snr = dict.fromkeys(AB_MAGS)
times_to_reach_snrs = {snr: times_to_one_snr.copy() for snr in TARGET_SNRS}

for ab_mag in AB_MAGS:
    #
    # Initialize a point source and generate a flat spectrum in units of AB magnitude
    #
    MySource = PointSource()
    MySource.generate_uniform(wavelengths=wavelengths, value=ab_mag, unit="ABmag")
    #
    # Initialize a Photometry object and use an optimal aperture for a point source
    #
    MyPhot = Photometry(MyTelescope, MySource, MyBackground)
    MyPhot.use_optimal_aperture(quiet=True)  # don't print encircled energy fraction
    for snr in TARGET_SNRS:
        #
        # Find the time, in seconds, required to reach the specified signal-to-noise ratio
        #
        times_to_reach_snrs[snr][ab_mag] = MyPhot.calc_snr_or_t(
            snr=snr, reddening=REDDENING, quiet=True
        )

In [4]:
for snr in TARGET_SNRS:
    print(f"Times (s) required to reach S/N = {snr}")
    display(pd.DataFrame(times_to_reach_snrs[snr]).round(2).T)

Times (s) required to reach S/N = 5


,uv,u,g
22.0,5.01,5.19,4.29
22.5,7.94,8.23,6.84
23.0,12.60,13.07,10.93
23.5,19.99,20.77,17.56
24.0,31.73,33.08,28.45
24.5,50.43,52.83,46.66
25.0,80.27,84.74,77.97
25.5,128.10,136.88,134.03
26.0,205.24,223.47,239.86
26.5,330.91,370.93,452.78


Times (s) required to reach S/N = 10


,uv,u,g
22.0,14.27,14.78,12.29
22.5,22.63,23.48,19.65
23.0,35.90,37.33,31.60
23.5,57.01,59.46,51.23
24.0,90.61,94.99,84.09
24.5,144.26,152.44,140.66
25.0,230.29,246.39,241.84
25.5,369.15,402.64,431.94
26.0,595.57,669.00,810.18
26.5,970.51,1139.13,1608.42
